# NEAR Project Visualization


Visualize , gaze point, fixation, and blink

Mount file to colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Heatmap and Gaze point

In [ ]:
import imageio.v2 as imageio
from collections import Counter
from PIL import Image
import numpy as np
import os, glob, shutil, re
from collections import defaultdict
from pathlib import Path

# ---- task folders & Drive output root ----
# Corrected SRC_ROOT to point to the shared drive shortcut
SRC_ROOT = (
    "/content/drive/.shortcut-targets-by-id/"
    "1BH7hdSj4Nh64k4RfY-YVSzgVZm29RTrL"
    "/NEAR_Experiment_Design/PilotData_V1_10232025"
)
DRIVE_ROOT = "/content/drive/MyDrive/NEAR_Experiment_Design/PilotData_V1_10232025/1_Data_Analysis/heatmap_gazepoint"

# Subject -> list of task folder names under SRC_ROOT
WINDOW_SEC = 3.0
TASKS = {
    "AT":  ["AT_1", "AT_2", "AT_3_1", "AT_3_2"],
    "Ayu": ["Ayu_1", "Ayu_2", "Ayu_3"],
    "JC":  ["JC_1", "JC_2", "JC_3_1", "JC_3_2"],
    "KC":  ["KC_1", "KC_2", "KC_3_1", "KC_3_2"],
    "LKH": ["LKH_1", "LKH_2", "LKH_3_1", "LKH_3_2"],
    "SYH": ["SYH_1_simple", "SYH_2_simple", "SYH_3_1_simple", "SYH_3_2_simple"],
    "YL":  ["YL_1", "YL_2", "YL_3"],
}

# ---- utilities ----
def sort_by_window(files):
    """Sort filenames like AT_000-005s_heat.png by window start (000)."""
    def key_fn(p):
        m = re.search(r"_(\d+)-\d+s", os.path.basename(p))
        return int(m.group(1)) if m else 10**9
    return sorted(files, key=key_fn)

def to_even_size(size):
    """Make (W,H) even to avoid H.264 issues."""
    w, h = size
    w = w - (w % 2)
    h = h - (h % 2)
    # guard minimums
    w = max(2, w); h = max(2, h)
    return (w, h)

def ensure_dirs(subject):
    """Create subject-specific output folders on Drive."""
    os.makedirs(f"{DRIVE_ROOT}/heatmap/{subject}", exist_ok=True)
    os.makedirs(f"{DRIVE_ROOT}/gaze_point/{subject}", exist_ok=True)
    os.makedirs(f"{DRIVE_ROOT}/animation/{subject}", exist_ok=True)

def clear_local_output():
    """Clear local output folders before each task to avoid mixing frames."""
    for p in ["/content/heat_map", "/content/gaze_point"]:
        if os.path.exists(p):
            for f in glob.glob(os.path.join(p, "*")):
                try:
                    os.remove(f)
                except IsADirectoryError:
                    shutil.rmtree(f, ignore_errors=True)
        else:
            os.makedirs(p, exist_ok=True)

def copy_with_prefix(src_glob, dst_dir, prefix):
    """Copy files matching src_glob into dst_dir; prefix the filename with `prefix` to avoid collisions."""
    files = sorted(glob.glob(src_glob))
    if not files:
        print(f"[copy] No match: {src_glob}")
        return 0
    count = 0
    for s in files:
        base = os.path.basename(s)
        base_tail = base.split("_", 1)[1] if "_" in base else base
        new_name = f"{prefix}_{base_tail}"
        dst = os.path.join(dst_dir, new_name)
        shutil.copy2(s, dst)
        count += 1
    print(f"[copy] {count} files -> {dst_dir} (prefix={prefix})")
    return count

def make_mp4_from_folder(folder, pattern, out_mp4_path, fps=2):
    """
    Create an MP4 video from PNGs in `folder` matching `pattern`.
    Frames are converted to RGB and resized to a common even size.
    fps controls playback speed (lower = slower).
    """
    files = sort_by_window(glob.glob(os.path.join(folder, pattern)))
    print(f"[video] Found {len(files)} frames for {out_mp4_path}")
    if not files:
        print("[video] No frames, skipped:", out_mp4_path)
        return

    # decide target size using the most common WxH, then make it even
    sizes = []
    for p in files:
        with Image.open(p) as im:
            sizes.append(im.size)
    common_size = Counter(sizes).most_common(1)[0][0]
    target_size = to_even_size(common_size)
    print("[video] Target frame size:", target_size, "(from most common size:", common_size, ")")
    print("[video] FPS:", fps)

    # write mp4 using imageio-ffmpeg (libx264)
    # Note: quality=8 is a good balance; you can tweak if needed.
    with imageio.get_writer(out_mp4_path, fps=fps, codec="libx264", quality=8, macro_block_size=None) as writer:
        for p in files:
            im = Image.open(p).convert("RGB")
            if im.size != target_size:
                print(f"  [resize] {os.path.basename(p)} {im.size} -> {target_size}")
                im = im.resize(target_size, Image.LANCZOS)
            frame = np.array(im)  # RGB
            writer.append_data(frame)

    print("[video] Saved:", out_mp4_path)

# ---- per-task runner ----
def run_one_task(task_dir, subject, task_tag, window_sec=WINDOW_SEC):
    """
    Run your existing visualization pipeline for one task_dir.
    Assumes the earlier cell's functions are already defined in this runtime:
      - load_gaze_dataframe, open_world_video, plot_and_save_heatmap, plot_and_save_points
    Saves PNGs to /content/heat_map and /content/gaze_point, and MP4 to those folders as well.
    Then copies them to Drive, renaming with 'task_tag__' prefix to avoid collisions.
    """
    print("\n" + "="*80)
    print(f"[run] Subject={subject} | Task={task_tag} | Dir={task_dir}")
    print("="*80)

    # clean local outputs so frames belong only to this task
    clear_local_output()

    # ---- generate frames + mp4 using your existing functions ----
    # If you encapsulated your earlier logic as a function, just call it here (preferred).
    # Otherwise, reproduce the essential calls inline:

    # 1) Load gaze
    gaze_df = load_gaze_dataframe(task_dir)
    print("[gaze_df] shape:", gaze_df.shape)

    # 2) Open video
    cap, fps, frame_count, duration, W, H = open_world_video(task_dir)

    # 3) Windowing and per-window plots (same as your earlier cell)
    WINDOW_SEC = float(window_sec)
    t_min = float(gaze_df["timestamp"].min())
    gaze_df = gaze_df.assign(t_rel=gaze_df["timestamp"] - t_min)
    t_max_rel = float(gaze_df["t_rel"].max())
    max_duration = min(duration, t_max_rel)
    num_windows = int(max_duration // WINDOW_SEC) + 1
    print(f"[time] 0..{max_duration:.2f}s | window={WINDOW_SEC}s | windows≈{num_windows}")

    for k in range(num_windows):
        start_t = k * WINDOW_SEC
        end_t = min((k + 1) * WINDOW_SEC, max_duration)
        if end_t <= start_t + 1e-6:
            continue
        win_mask = (gaze_df["t_rel"] >= start_t) & (gaze_df["t_rel"] < end_t)
        df_win = gaze_df.loc[win_mask]
        if df_win.empty:
            continue

        # mid frame
        mid_t = 0.5 * (start_t + end_t)
        frame_bgr = grab_frame_at_time(cap, fps, mid_t)
        if frame_bgr is None:
            continue
        import cv2
        frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)

        # filenames (keep your original SUBJECT-only names here)
        tag = f"{int(start_t):03d}-{int(end_t):03d}s"
        heat_path = f"/content/heat_map/{subject}_{tag}_heat.png"
        pts_path  = f"/content/gaze_point/{subject}_{tag}_points.png"

        plot_and_save_heatmap(frame_rgb, df_win, heat_path)
        plot_and_save_points(frame_rgb, df_win, pts_path)

    cap.release()

    # 4) Build MP4s from the frames
    make_mp4_from_folder("/content/heat_map",  f"{subject}_*-*s_heat.png",
                         f"/content/heat_map/{subject}_heatmap.mp4", fps=2)
    make_mp4_from_folder("/content/gaze_point", f"{subject}_*-*s_points.png",
                         f"/content/gaze_point/{subject}_gaze_points.mp4", fps=2)


    # ---- copy to Drive, prefixing with task tag to avoid collisions ----
    ensure_dirs(subject)
    copy_with_prefix("/content/heat_map/*.png",   f"{DRIVE_ROOT}/heatmap/{subject}",   prefix=task_tag)
    copy_with_prefix("/content/gaze_point/*.png", f"{DRIVE_ROOT}/gaze_point/{subject}",prefix=task_tag)

    # MP4s → animation
    mp4_count  = copy_with_prefix("/content/heat_map/*.mp4",   f"{DRIVE_ROOT}/animation/{subject}", prefix=task_tag)
    mp4_count += copy_with_prefix("/content/gaze_point/*.mp4", f"{DRIVE_ROOT}/animation/{subject}", prefix=task_tag)
    if mp4_count == 0:
        print("[note] No MP4s for this task (ok).")

# ---- Run ----
for subject, task_list in TASKS.items():
    for task_tag in task_list:
        task_dir = os.path.join(SRC_ROOT, task_tag)
        if not os.path.isdir(task_dir):
            print(f"[skip] Task dir not found: {task_dir}")
            continue
        try:
            run_one_task(task_dir, subject=subject, task_tag=task_tag, window_sec=WINDOW_SEC)
        except Exception as e:
            print(f"[ERROR] {subject}:{task_tag} -> {e}")


[run] Subject=AT | Task=AT_1 | Dir=/content/drive/.shortcut-targets-by-id/1BH7hdSj4Nh64k4RfY-YVSzgVZm29RTrL/NEAR_Experiment_Design/PilotData_V1_10232025/AT_1
[exports] Loading: /content/drive/.shortcut-targets-by-id/1BH7hdSj4Nh64k4RfY-YVSzgVZm29RTrL/NEAR_Experiment_Design/PilotData_V1_10232025/AT_1/exports/000/gaze_positions.csv
[exports] Columns: ['gaze_timestamp', 'world_index', 'confidence', 'norm_pos_x', 'norm_pos_y', 'base_data', 'gaze_point_3d_x', 'gaze_point_3d_y', 'gaze_point_3d_z', 'eye_center0_3d_x', 'eye_center0_3d_y', 'eye_center0_3d_z', 'gaze_normal0_x', 'gaze_normal0_y', 'gaze_normal0_z', 'eye_center1_3d_x', 'eye_center1_3d_y', 'eye_center1_3d_z', 'gaze_normal1_x', 'gaze_normal1_y', 'gaze_normal1_z']
[exports] norm_pos outside [0,1]; applying auto-normalization
[exports] Using timestamp: gaze_timestamp | positions from normalized columns: norm_pos_x,norm_pos_y
      timestamp  norm_pos_x  norm_pos_y  confidence
0  66047.764624    0.484016    0.000000         0.0
1  66047

## Fixation Trajectory

In [ ]:
import os
import math
import numpy as np
import pandas as pd
import cv2
import imageio

SRC_ROOT = (
    "/content/drive/.shortcut-targets-by-id/"
    "1BH7hdSj4Nh64k4RfY-YVSzgVZm29RTrL"
    "/NEAR_Experiment_Design/PilotData_V1_10232025"
)

# Output folder in your MyDrive
OUTPUT_DIR = "/content/drive/MyDrive/NEAR_fixation_Trajectory_Output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

INTERVAL_SEC = 3.0
GIF_FRAME_DURATION = 0.3
MIN_FIX_PER_FRAME = 1

print("Reading from (shared):", SRC_ROOT)
print("Writing outputs to:", OUTPUT_DIR)

def load_fixations(exports_dir):
    df = pd.read_csv(os.path.join(exports_dir, "fixations.csv"))
    ts_col = next((c for c in ["start_timestamp","timestamp","world_timestamp"] if c in df.columns), None)
    x_col = next((c for c in ["norm_pos_x","x","gaze_point_2d_x"] if c in df.columns), None)
    y_col = next((c for c in ["norm_pos_y","y","gaze_point_2d_y"] if c in df.columns), None)
    df = df[[ts_col, x_col, y_col]].rename(columns={ts_col:"t", x_col:"x", y_col:"y"})
    df = df.sort_values("t").reset_index(drop=True)
    return df

def load_bg(exports_dir):
    cap = cv2.VideoCapture(os.path.join(exports_dir, "world.mp4"))
    ok, frame = cap.read()
    cap.release()
    if not ok or frame is None:
        frame = np.ones((720,1280,3), dtype=np.uint8)*255
    return frame

def draw_traj(bg, df, normalize=True):
    img = bg.copy()
    h, w = img.shape[:2]
    xs = df["x"].to_numpy(float)
    ys = df["y"].to_numpy(float)
    if normalize:
        xs = np.clip(xs, 0, 1)*w
        ys = (1 - np.clip(ys, 0, 1))*h
    pts = list(zip(xs.astype(int), ys.astype(int)))
    if not pts:
        return img
    for (x1,y1),(x2,y2) in zip(pts[:-1], pts[1:]):
        cv2.line(img, (x1,y1), (x2,y2), (255,0,255), 2)
    for x,y in pts:
        cv2.circle(img, (x,y), 6, (0,255,0), -1)
    return img

# Iterate over all subjects/tasks in shared root
for subj in sorted(os.listdir(SRC_ROOT)):
    subject_dir = os.path.join(SRC_ROOT, subj)
    exports000 = os.path.join(subject_dir, "exports", "000")
    if not os.path.isdir(exports000):
        print("[SKIP-NO-DATA] ", subj)
        continue

    print("\nProcessing:", subj)
    try:
        fix_df = load_fixations(exports000)
    except Exception as e:
        print("  [ERROR load fixations]", e)
        continue

    # ADDED: Check if fix_df is empty before proceeding
    if fix_df.empty:
        print("  [SKIP] No fixations data available for", subj)
        continue

    bg = load_bg(exports000)
    normalize = ((fix_df["x"].between(0,1)).mean() > 0.5)

    t0, t1 = fix_df["t"].min(), fix_df["t"].max()
    total = t1 - t0
    n_int = int(math.ceil(total / INTERVAL_SEC))
    print(f"  Duration {total:.1f}s, {len(fix_df)} fixations → {n_int} windows")

    # static windows
    for i in range(n_int):
        ws = t0 + i*INTERVAL_SEC
        we = min(ws + INTERVAL_SEC, t1)
        dfw = fix_df[(fix_df["t"]>=ws)&(fix_df["t"]<we)]
        if dfw.empty: continue
        img = draw_traj(bg, dfw, normalize)
        label = f"{ws - t0:.1f}-{we - t0:.1f}s".replace('.', 'p')
        out_fn = f"{subj}_fixtraj_{label}.png"
        cv2.imwrite(os.path.join(OUTPUT_DIR, out_fn), img)

    # full-trajectory GIF
    frames = []
    for i in range(0, len(fix_df), MIN_FIX_PER_FRAME):
        dfp = fix_df.iloc[:i+1]
        img = draw_traj(bg, dfp, normalize)
        frames.append(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    if frames:
        gif_path = os.path.join(OUTPUT_DIR, f"{subj}_fixtraj_full.gif")
        imageio.mimsave(gif_path, frames, duration=GIF_FRAME_DURATION)

    print("  → done", subj)

print("\nFinished. All outputs in:", OUTPUT_DIR)


Reading from (shared): /content/drive/.shortcut-targets-by-id/1BH7hdSj4Nh64k4RfY-YVSzgVZm29RTrL/NEAR_Experiment_Design/PilotData_V1_10232025
Writing outputs to: /content/drive/MyDrive/NEAR_fixation_Trajectory_Output
[SKIP-NO-DATA]  .ipynb_checkpoints
[SKIP-NO-DATA]  1_Data_Analysis

Processing: AT_1
  Duration 52.4s, 28 fixations → 18 windows
  → done AT_1

Processing: AT_2
  Duration 19.1s, 10 fixations → 7 windows
  → done AT_2

Processing: AT_3_1
  Duration 7.7s, 8 fixations → 3 windows
  → done AT_3_1

Processing: AT_3_2
  Duration 33.6s, 23 fixations → 12 windows
  → done AT_3_2

Processing: Ayu_1
  Duration 52.6s, 88 fixations → 18 windows
  → done Ayu_1

Processing: Ayu_2
  Duration 31.0s, 26 fixations → 11 windows
  → done Ayu_2

Processing: Ayu_3
  Duration 76.9s, 48 fixations → 26 windows
  → done Ayu_3

Processing: JC_1
  Duration 49.2s, 50 fixations → 17 windows
  → done JC_1

Processing: JC_2
  [SKIP] No fixations data available for JC_2

Processing: JC_3_1
  Duration 19.1

## Blink Pupil

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import file_methods as fm
import helper_funcs as hf

# -----------------------------
# Paths & config
# -----------------------------
SRC_ROOT = (
    "/content/drive/.shortcut-targets-by-id/"
    "1BH7hdSj4Nh64k4RfY-YVSzgVZm29RTrL"
    "/NEAR_Experiment_Design/PilotData_V1_10232025"
)

SAVE_ROOT = ("/content/drive/MyDrive/NEAR_Experiment_Design/PilotData_V1_10232025/1_Data_Analysis/blink_pupil")

WINDOW_SEC = 3.0

TASK_DICT = {
    "AT":  ["AT_1", "AT_2", "AT_3_1", "AT_3_2"],
    "Ayu": ["Ayu_1", "Ayu_2", "Ayu_3"],
    "JC":  ["JC_1", "JC_2", "JC_3_1", "JC_3_2"],
    "KC":  ["KC_1", "KC_2", "KC_3_1", "KC_3_2"],
    "LKH": ["LKH_1", "LKH_2", "LKH_3_1", "LKH_3_2"],
    "SYH": ["SYH_1_simple", "SYH_2_simple", "SYH_3_1_simple", "SYH_3_2_simple"],
    "YL":  ["YL_1", "YL_2", "YL_3"],
}

# -----------------------------
# Main loop
# -----------------------------
for subject, task_list in TASK_DICT.items():

    for task in task_list:
        print(f"Processing {subject} | {task}")

        task_path = os.path.join(SRC_ROOT, task)
        save_task_dir = os.path.join(SAVE_ROOT, task)
        os.makedirs(save_task_dir, exist_ok=True)

        # ---- Load timestamps ----
        timest = np.load(os.path.join(task_path, "world_timestamps.npy"))

        # ---- Load pupil PLData ----
        cols = ['id', 'timestamp', 'topic', 'confidence',
                'norm_pos', 'diameter', 'diameter_3d']

        pupil_df = pl_to_df('pupil', task_path, cols)

        pupil_df[['norm_pos_x', 'norm_pos_y']] = pd.DataFrame(
            pupil_df['norm_pos'].tolist(), index=pupil_df.index
        )
        pupil_df.drop(columns='norm_pos', inplace=True)

        pupil_df.rename(columns={
            'confidence': 'conf',
            'diameter': 'dia_2d',
            'diameter_3d': 'dia_3d'
        }, inplace=True)

        pupil_df['seconds'] = pupil_df['timestamp'] - timest[0]
        pupil_df = pupil_df[pupil_df['topic'].str.contains('2d')]

        # ---- Windowing ----
        max_t = pupil_df['seconds'].max()
        n_windows = int(np.ceil(max_t / WINDOW_SEC))

        for w in range(n_windows):
            t_start = w * WINDOW_SEC
            t_end = t_start + WINDOW_SEC

            win_df = pupil_df[
                (pupil_df['seconds'] >= t_start) &
                (pupil_df['seconds'] < t_end)
            ]

            if win_df.empty:
                continue

            right = win_df[win_df['id'] == 0]
            left  = win_df[win_df['id'] == 1]

            # ---- Plot ----
            fig, axes = plt.subplots(1, 2, figsize=(10, 5), sharey=True)

            axes[0].plot(right['seconds'], right['dia_2d'])
            axes[0].set_title("Eye 0", fontsize=18)
            axes[0].set_xlabel("time (s)", fontsize=18)
            axes[0].set_ylabel("diameter (mm)", fontsize=18)
            axes[0].tick_params(labelsize=16)
            axes[0].set_box_aspect(1)

            axes[1].plot(left['seconds'], left['dia_2d'])
            axes[1].set_title("Eye 1", fontsize=18)
            axes[1].set_xlabel("time (s)", fontsize=18)
            axes[1].tick_params(labelsize=16)
            axes[1].set_box_aspect(1)

            plt.tight_layout()

            # ---- Save ----
            fname = f"{task}_{t_start:.1f}-{t_end:.1f}s.png"
            plt.savefig(os.path.join(save_task_dir, fname), dpi=300)
            plt.close(fig)

Processing AT | AT_1
Processing AT | AT_2
Processing AT | AT_3_1
Processing AT | AT_3_2
Processing Ayu | Ayu_1
Processing Ayu | Ayu_2
Processing Ayu | Ayu_3
Processing JC | JC_1
Processing JC | JC_2
Processing JC | JC_3_1
Processing JC | JC_3_2
Processing KC | KC_1
Processing KC | KC_2
Processing KC | KC_3_1
Processing KC | KC_3_2
Processing LKH | LKH_1
Processing LKH | LKH_2
Processing LKH | LKH_3_1
Processing LKH | LKH_3_2
Processing SYH | SYH_1_simple
Processing SYH | SYH_2_simple
Processing SYH | SYH_3_1_simple
Processing SYH | SYH_3_2_simple
Processing YL | YL_1
Processing YL | YL_2
Processing YL | YL_3
